In [ ]:
import os
import zipfile
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from skimage.transform import resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Mount Google Drive and extract files
from google.colab import drive
drive.mount('/content/drive')

# Path to the zip file
zip_path = '/content/drive/MyDrive/mahak/mahak.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/mahak')

# Load images from DICOM files
def load_images(patient_folder):
    ct_images, pet_images = [], []
    ct_folder = None
    pet_folder = None

    # Find CT and PET folders
    for folder in os.listdir(patient_folder):
        if 'CT' in folder:
            ct_folder = os.path.join(patient_folder, folder)
        elif 'PET' in folder:
            pet_folder = os.path.join(patient_folder, folder)

    # If CT or PET folder is not found, return empty arrays
    if ct_folder is None or pet_folder is None:
        print(f"CT or PET folder not found for {patient_folder}")
        return np.array(ct_images), np.array(pet_images)

    # Load images from found CT and PET folders
    for filename in sorted(os.listdir(ct_folder)):
        if filename.endswith(".dcm"):
            ct_image_path = os.path.join(ct_folder, filename)
            pet_image_path = os.path.join(pet_folder, filename)

            ct_image = pydicom.dcmread(ct_image_path).pixel_array
            pet_image = pydicom.dcmread(pet_image_path).pixel_array

            # Resize images to 256x256
            ct_image = resize(ct_image, (256, 256), anti_aliasing=True)
            pet_image = resize(pet_image, (256, 256), anti_aliasing=True)

            ct_images.append(ct_image)
            pet_images.append(pet_image)

    return np.array(ct_images), np.array(pet_images)

# Load data for all patients
patient_folders = [os.path.join('/content/mahak', folder) for folder in os.listdir('/content/mahak')]
ct_data, pet_data = [], []
for folder in patient_folders:
    ct, pet = load_images(folder)
    ct_data.append(ct)
    pet_data.append(pet)

# Convert to numpy arrays, filtering out empty arrays
ct_data = np.concatenate([data for data in ct_data if data.size > 0], axis=0)
pet_data = np.concatenate([data for data in pet_data if data.size > 0], axis=0)

# Split into training and test sets
ct_train, ct_test, pet_train, pet_test = train_test_split(ct_data, pet_data, test_size=0.2, random_state=42)

# Normalize pixel values to [0, 1]
ct_train = ct_train[..., np.newaxis] / 255.0
pet_train = pet_train[..., np.newaxis] / 255.0
ct_test = ct_test[..., np.newaxis] / 255.0
pet_test = pet_test[..., np.newaxis] / 255.0

# Data augmentation
data_gen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=20)

# Build U-Net model
def unet_model(input_size=(256, 256, 1)):
    inputs = Input(input_size)

    # Down-sampling path
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(p3)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(p4)

    # Up-sampling path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(u6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(u7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', padding='same')(u8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(16, (3, 3), activation='relu', padding='same')(u9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Compile model
model = unet_model(input_size=(256, 256, 1))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train model with data augmentation
model.fit(data_gen.flow(ct_train, pet_train, batch_size=4),
          steps_per_epoch=len(ct_train) // 4,
          epochs=10,
          validation_data=(ct_test, pet_test))

# Perform registration on the test set
test_patient_index = 0  # Choose the first patient from the test set
ct_test_patient = ct_test[test_patient_index]

# Register each slice
registered_images = []
for slice_idx in range(ct_test_patient.shape[0]):
    ct_slice = ct_test_patient[slice_idx][np.newaxis, ..., np.newaxis]  # Correct shape: (1, 256, 256, 1)
    registered_image = model.predict(ct_slice)
    registered_images.append(registered_image[0, ..., 0])

# Save results
save_folder = '/content/mahak/registration_results'
os.makedirs(save_folder, exist_ok=True)
for i, img in enumerate(registered_images):
    plt.imsave(f"{save_folder}/registered_slice_{i}.png", img, cmap='gray')

print(f"Registration complete. Results saved in {save_folder}")
